In [1]:
!pip install -q PyPDF2
!pip install fitz
!pip install PyMuPDF
!pip install bert-extractive-summarizer
!pip install -q tabula-py
import PyPDF2
import re
import pandas as pd
import fitz
from summarizer import Summarizer
import tabula

     |████████████████████████████████| 81kB 2.5MB/s 
     |████████████████████████████████| 3.1MB 3.8MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 235kB 27.6MB/s 
     |████████████████████████████████| 450kB 22.6MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
     |████████████████████████████████| 430kB 26.7MB/s 
     |████████████████████████████████| 5.8MB 34.9MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 348kB 41.1MB/s 
  Created wheel for pyxnat: filename=pyxnat-1.2.1.0.post3-cp36-none-any.whl size=71623 sha256=8f89c364334eee058416522a82da9b035be68fc678c15e92e392a2519ce06d5c
  Stored in directory: /root/.cache/pip/wheels/98/46/71/7096c8f1537087e7628bdcc723f6e766880f8dde2667009371
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=0c9de3aaab67dad2ec5abb874bc2db81abc3a43262122bd7b3b8fec81134fb6b
  Stored in directory: /ro

In [0]:
path = '/content/MM Project.pdf'
index = ['Steeple Analysis',
        ['Social', 'Technological', 'Economic', 'Ethical', 'Political', 'Legal', 'Environmental'],
        'Swot Analysis',
        ['Strengths', 'Weakenesses', 'Opportunities', 'Threats'],
        'Porters 5 forces model',
        ['Force 1', 'Force 2', 'Force 3', 'Force 4', 'Force 5'],
        'Value Chain Analysis',
        ['Primary Activities', 'Secondary Activities'],
        'Mission',
        [],
        'Vision',
        [],
        'Goals',
        [],
        'Objectives',
        [],
        'Consumer Behaviour',
        ['Survey', 'Cleaning Data', 'Exploratory Data Analysis'],
        'Marketing Mix',
        ['Product', 'Promotions', 'Price', 'Place'],
        'Segmentation, Targeting, Positioning, Differentiation',
        ['Segmentation', 'Targeting', 'Positioning', 'Differentiation'],
        'Marketing Budget',
        ['Pre Launch', 'Launch', 'Post Launch']]
pages= [3,3,3,4,4,5,5,7,8,9,10,11,12,12,13,14,15,17,18,19,19,19,20,21,
        25,30,31,32,32,32,35,37,38,40,40,41]

In [0]:
def read_pdf(path):
  info = []
  with open(path, 'rb') as fil:
    pdf = PyPDF2.PdfFileReader(fil)
    pages = pdf.getNumPages()
    for pageno in range(pages):
      page = pdf.getPage(pageno)
      content = page.extractText()
      info.append(content)
  cont = ''
  for i in range(3, len(info)):
    cont+=info[i]
  cont = cont.replace('\n', ' ')
  return info, cont

In [0]:
def get_titles(content):
  topics = content.split('\n \n')
  topics = [x.replace('\n', '') for x in topics]
  topics = [x for x in topics if x.isalpha()]
  return topic

In [0]:
def make_df(list1, list2):
  result = []
  k = 0
  for i in list1:
    if type(i) == str:
      topic = i
    else:
      if i:
        for j in range(len(i)):        
          result.append([topic, i[j], list2[k]])
          k+=1
      else:
        result.append([topic, topic, list2[k]])
        k+=1
  df = pd.DataFrame(result, columns=['topics', 'subtopics', 'page_no'])
  return df

In [0]:
def extract_text(column_name):
  text = []
  df = make_df(index, pages)
  _, cont = read_pdf(path)
  for i in range(df.shape[0]):
    sub1 = df[column_name][i]
    try:    
      sub2 = df[column_name][i+1]
    except KeyError:
      sub2 = 'lol'
    if sub2 not in cont.split(sub1, 1)[-1]:
      text.append('')
    else:
      text.append(cont.split(sub1, 1)[-1].split(sub2, 1)[0])
  return text

In [0]:
def extract_summary(topic):
  s = Summarizer()
  res = s(topic, min_length=10)
  result = ''.join(res)
  return result

In [0]:
def extract_table(topic):
  df = make_df(index, pages)
  info ,_ = read_pdf(path)
  #file_name = '/content/MM Project.pdf'
  page_start= int(df.loc[df['subtopics'] == topic, 'page_no'])+1
  ind= int(df[df['subtopics']==topic].index.values)
  ind+=1
  #print(page_start)
  try:
    page_end= int(df.loc[df['subtopics'] == df['subtopics'][ind], 'page_no'])
    #print(page_end)
  except KeyError:
    page_end= len(info) #change to len(info) when code fancy
  tables= []
  for page in (page_start, page_end):
      table = tabula.read_pdf(path, pages = page, multiple_tables = True)
      tables.append(table)
  return len(tables), tables

In [0]:
def extract_image(topic):  
  df = make_df(index, pages)
  doc= fitz.open(path)
  page_start= int(df.loc[df['subtopics'] == topic, 'page_no'])+1
  ind= int(df[df['subtopics']==topic].index.values)
  ind+=1
  #print(page_start)
  page_end= int(df.loc[df['subtopics'] == df['subtopics'][ind], 'page_no'])+1
  #print(page_end)
  for page in (page_start, page_end):
      for img in doc.getPageImageList(page):
          xref = img[0]
          pix = fitz.Pixmap(doc, xref)
          if pix.n < 5:       # this is GRAY or RGB
              pix.writePNG("p%s-%s.png" % (page, xref))
          else:               # CMYK: convert to RGB first
              pix1 = fitz.Pixmap(fitz.csRGB, pix)
              pix1.writePNG("p%s-%s.png" % (page, xref))
              pix1 = None
          pix = None
  #return pix1

In [0]:
df = make_df(index, pages)

In [0]:
df['subtopics'].apply(extract_image)

In [0]:
df['texts'] = extract_text('subtopics')

In [11]:
df['tables'] = df['subtopics'].apply(extract_table)

Apr 06, 2020 8:46:54 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Apr 06, 2020 8:46:54 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>



In [12]:
df['summary'] = df['texts'].apply(extract_summary)

In [15]:
df

,topics,subtopics,page_no,texts,tables,summary
0,Steeple Analysis,Social,3,Considering the social circumstances is ...,"(2, [[], []])",Considering the social circumstances is essent...
1,Steeple Analysis,Technological,3,The rapid technological advancement acro...,"(2, [[], []])",The rapid technological advancement across the...
2,Steeple Analysis,Economic,3,It is important for Nykaa to understand ...,"(2, [[], []])",It is important for Nykaa to understand econom...
3,Steeple Analysis,Ethical,4,Ethics are codes which are necessary to ...,"(2, [[], []])",Ethics are codes which are necessary to run th...
4,Steeple Analysis,Political,4,Political factors exert a strong influen...,"(2, [[], []])",Political factors exert a strong influence on ...
5,Steeple Analysis,Legal,5,Nykaa cannot enter a new market without ...,"(2, [[Empty DataFrame\nColumns: [●, Employee p...",Nykaa cannot enter a new market without studyi...
6,Steeple Analysis,Environmental,5,The growing environme ntal pollution cou...,"(2, [[Empty DataFrame\nColumns: [●, Employee p...",The growing environme ntal pollution coupled w...
7,Swot Analysis,Strengths,7,,"(2, [[Empty DataFrame\nColumns: [S1, 1]\nIndex...",
8,Swot Analysis,Weakenesses,8,4 STEEPLE Analysis 1. Social Cons...,"(2, [[ Unnamed: 0 ... ...",4 STEEPLE Analysis 1. Demographics trends:...
9,Swot Analysis,Opportunities,9,O1 Exponential growth in the population: ...,"(2, [[ Unnamed: 0 Exponential g...",O1 Exponential growth in the population: ...


In [0]:
df.to_csv('final.csv')